In [1]:
import pandas as pd

In [3]:
fake_data=pd.read_csv("Fake.csv")

fake_data['result'] =1

In [ ]:
fake_data.head()
fake_data.to_csv("Fake_with_result.csv", index=False)

In [6]:
fake_data=pd.read_csv("True.csv")

fake_data['result'] =0
fake_data.head()
fake_data.to_csv("True_with_result.csv", index=False)

In [7]:


# Load both CSV files
df1 = pd.read_csv("Fake_with_result.csv")
df2 = pd.read_csv("True_with_result.csv")

# Vertically stack (concatenate) them
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv("Final_data.csv", index=False)

print("✅ Files stacked successfully.")
print("📁 Combined CSV saved as: combined_file.csv")


✅ Files stacked successfully.
📁 Combined CSV saved as: combined_file.csv


In [11]:
data=pd.read_csv("Final_data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   result   44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import joblib
import os

In [12]:
df = pd.read_csv("Final_data.csv")
X = df[['title', 'text', 'subject']]  # Input features
y = df['result']       

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('title_tfidf', TfidfVectorizer(stop_words='english', max_features=1000), 'title'),
    ('text_tfidf', TfidfVectorizer(stop_words='english', max_features=3000), 'text'),
    ('subject_ohe', OneHotEncoder(handle_unknown='ignore'), ['subject']),
])


In [17]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Naive Bayes": MultinomialNB(),
    "Support Vector Machine": SVC(probability=True),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [18]:
# 🔹 Create results list and save path
results = []
save_path = "saved_models"
os.makedirs(save_path, exist_ok=True)

# 🔹 Train, evaluate, save models
for name, model in models.items():
    print(f"\n🚀 Training: {name}")
    
    # Full pipeline
    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('clf', model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    print(f"✅ Accuracy: {acc:.4f}")
    print("📊 Classification Report:")
    print(classification_report(y_test, y_pred))
    
    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision (0)": report['0']['precision'],
        "Recall (0)": report['0']['recall'],
        "F1-score (0)": report['0']['f1-score'],
        "Precision (1)": report['1']['precision'],
        "Recall (1)": report['1']['recall'],
        "F1-score (1)": report['1']['f1-score'],
    })
    
    # Save the model
    model_filename = os.path.join(save_path, f"{name.replace(' ', '_').lower()}.pkl")
    joblib.dump(pipeline, model_filename)
    print(f"💾 Model saved to: {model_filename}")



🚀 Training: Logistic Regression
✅ Accuracy: 1.0000
📊 Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4247
           1       1.00      1.00      1.00      4733

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

💾 Model saved to: saved_models\logistic_regression.pkl

🚀 Training: Random Forest
✅ Accuracy: 0.9996
📊 Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4247
           1       1.00      1.00      1.00      4733

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

💾 Model saved to: saved_models\random_forest.pkl

🚀 Training: Gradient Boosting
✅ Accuracy: 1.0000
📊 Classification Report:
              preci

c:\Users\athun\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [16:56:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Accuracy: 1.0000
📊 Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4247
           1       1.00      1.00      1.00      4733

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

💾 Model saved to: saved_models\xgboost.pkl


In [19]:

# 🔹 Show comparison table
results_df = pd.DataFrame(results)
print("\n📈 Final Model Comparison Table:")
print(results_df.to_string(index=False))


📈 Final Model Comparison Table:
                 Model  Accuracy  Precision (0)  Recall (0)  F1-score (0)  Precision (1)  Recall (1)  F1-score (1)
   Logistic Regression  1.000000       1.000000         1.0      1.000000            1.0    1.000000      1.000000
         Random Forest  0.999555       0.999059         1.0      0.999529            1.0    0.999155      0.999577
     Gradient Boosting  1.000000       1.000000         1.0      1.000000            1.0    1.000000      1.000000
           Naive Bayes  0.999555       0.999059         1.0      0.999529            1.0    0.999155      0.999577
Support Vector Machine  1.000000       1.000000         1.0      1.000000            1.0    1.000000      1.000000
   K-Nearest Neighbors  1.000000       1.000000         1.0      1.000000            1.0    1.000000      1.000000
               XGBoost  1.000000       1.000000         1.0      1.000000            1.0    1.000000      1.000000
